#### Deep Learning
#### Autores: Mariana David 
#### Autores: Pablo Escobar 
#### Laboratorio 6 y 7

# Red neuronal basado en el filtro colaborativo


In [18]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Load the dataset
ratings_df = pd.read_csv('./archive/Ratings.csv', sep=',')

user_label_encoder = LabelEncoder()
book_label_encoder = LabelEncoder()

ratings_df['Encoded_User_ID'] = user_label_encoder.fit_transform(ratings_df['User-ID'])
ratings_df['Encoded_Book_ID'] = book_label_encoder.fit_transform(ratings_df['ISBN'])
train_data, test_data = train_test_split(ratings_df, test_size=0.2, random_state=42)
embedding_dim = 32

num_users = ratings_df['Encoded_User_ID'].nunique()
num_books = ratings_df['Encoded_Book_ID'].nunique()

book_input = Input(shape=(1,))
book_embedding = Embedding(num_books, embedding_dim, input_length=1)(book_input)
book_vector = Flatten()(book_embedding)

user_input = Input(shape=(1,))
user_embedding = Embedding(num_users, embedding_dim, input_length=1)(user_input)
user_vector = Flatten()(user_embedding)

concatenated_vectors = Concatenate()([book_vector, user_vector])

dense1 = Dense(64, activation='relu')(concatenated_vectors)
dense2 = Dense(32, activation='relu')(dense1)
output = Dense(1)(dense2)

optimizer = Adam(learning_rate=0.001)

model = Model([user_input, book_input], output)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Display model summary
model.summary()


Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 input_10 (InputLayer)       [(None, 1)]                  0         []                            
                                                                                                  
 embedding_8 (Embedding)     (None, 1, 32)                1089779   ['input_9[0][0]']             
                                                          2                                       
                                                                                                  
 embedding_9 (Embedding)     (None, 1, 32)                3369056   ['input_10[0][0]']     

In [21]:
model.fit([train_data['Encoded_User_ID'], train_data['Encoded_Book_ID']], train_data['Book-Rating'], batch_size=128, epochs=10, verbose=1)

Epoch 1/10
  66/3594 [..............................] - ETA: 11:05 - loss: 12.5836